<a href="https://colab.research.google.com/github/utamiu1807/-utami-creditcardclustering/blob/main/Utami_Minilab_2_Factor_Reduction_and_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minilab 2: Factor Reduction and Feature Engineering

[Link to Data Dictionary](https://www.dropbox.com/scl/fi/iyyisxdp2rhij7204thsj/real_estate_data_dictionary.txt?rlkey=6h5ag5sctd6gon0otyti0w2tp&st=mb6cybb4&dl=1")

In [60]:
!pip install feature-engine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from google.colab import drive
from google.colab import files

In [61]:
# Real Estate Data
df = pd.read_csv('https://www.dropbox.com/scl/fi/vqm4cpjfzafaxpbb5h347/real_estate_estate_market_data_raw.csv?rlkey=8mcn2go4p8f10pwv3p59hrt47&st=jhftnxy8&dl=1')

df

,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,...,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Sale_Price,Longitude,Latitude
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141.0,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,NaN,0,5,2010,WD,Normal,215000,-93.619754,42.054035
1,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80.0,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,Minimum_Privacy,NaN,0,6,2010,WD,Normal,105000,-93.619756,42.053014
2,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81.0,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,Gar2,12500,6,2010,WD,Normal,172000,-93.619387,42.052659
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93.0,11160,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,...,No_Fence,NaN,0,4,2010,WD,Normal,244000,-93.617320,42.051245
4,Two_Story_1946_and_Newer,Residential_Low_Density,74.0,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,...,Minimum_Privacy,NaN,0,3,2010,WD,Normal,189900,-93.638933,42.060899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2824,Split_or_Multilevel,Residential_Low_Density,37.0,7937,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,CulDSac,...,Good_Privacy,NaN,0,3,2006,WD,Normal,142500,-93.604776,41.988964
2825,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,0.0,8885,Pave,No_Alley_Access,Slightly_Irregular,Low,AllPub,Inside,...,Minimum_Privacy,NaN,0,6,2006,WD,Normal,131000,-93.602680,41.988314
2826,Split_Foyer,Residential_Low_Density,62.0,10441,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,Minimum_Privacy,Shed,700,7,2006,WD,Normal,132000,-93.606847,41.986510
2827,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,77.0,10010,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,No_Fence,NaN,0,4,2006,WD,Normal,170000,-93.600190,41.990921


FYI How to select numeric and categorical columns

In [62]:
numerical_cols = df.select_dtypes(include=['number']).columns

categorical_cols = df.select_dtypes(include=['object', 'category']).columns

## Q1: Handling Missing Data

### Part A

In [63]:
# Look at documentation for df.isnull()
df.isna().any().any()

# Compute percent missing for each column
percent_missing = (df.isna().mean().sort_values(ascending=False))

# Keep only columns with missing values
percent_missing = percent_missing[percent_missing > 0]
df.columns[df.isna().any()]


Index(['MS_Zoning', 'Lot_Frontage', 'Lot_Config', 'House_Style',
       'Mas_Vnr_Type', 'Mas_Vnr_Area', 'Garage_Area', 'Misc_Feature'],
      dtype='object')

#### Part B

In [64]:
# Remove columns that are missing more than 15% of their values
columns_to_drop = percent_missing[percent_missing > 0.15].index.tolist()
df = df.drop(columns=columns_to_drop)

# display the percentage of missing values for the remaining columns
percent_missing_after_drop = df.isna().mean().sort_values(ascending=False)
percent_missing_after_drop = percent_missing_after_drop[percent_missing_after_drop > 0]
print(percent_missing_after_drop)

Lot_Frontage    0.147402
Garage_Area     0.139272
Mas_Vnr_Area    0.103217
MS_Zoning       0.065748
House_Style     0.063273
Lot_Config      0.061859
dtype: float64


#### Part C

In [65]:
zoning_col = "MS_Zoning"

# Replace missing values with the mode of the column
mode_value = df[zoning_col].mode()[0]
print(mode_value)
df[zoning_col] = df[zoning_col].fillna(mode_value)
df[zoning_col].value_counts().isna()

# replace other categorical columns with missing data
cat_cols = df.select_dtypes(include=["object", "category"]).columns
cat_cols_other = [c for c in cat_cols if c != zoning_col]  ## Exclude zoning column
df[cat_cols_other] = df[cat_cols_other].fillna("Missing")
df[cat_cols_other].isna().any().any()

Residential_Low_Density


np.False_

#### Part D

In [66]:
#impute all numeric columns with missing data
num_cols = df.select_dtypes(include=["int64", "float64"]).columns

for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

df.isna().any().any()

np.False_

#### Part E. : Another imputation method

Another method:
1) MICE (Multiple Imputation by Chained Equations)

package:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Explanation: MICE imputes missing values by iteratively modeling each variable with missing data as a function of the other variables, generating values that preserve the statistical relationships among features.

2) K-Nearest Neighbors (KNN) Imputation

Package: from sklearn.impute import KNNImputer

Explanation: KNN imputation fills missing values by averaging the values of the most similar observations (nearest neighbors), where similarity is defined by distance in feature space, allowing the imputed values to preserve local data structure.

In Python, both MICE and KNN imputation are primarily designed for numeric variables. KNN relies on distance calculations that require numeric features, while MICE in scikit-learn uses regression-based models and does not natively support categorical variables without specialized encoding and modeling.

## Question 2 Removing Outliers

#### Part A

In [67]:
#identify how many outliers the numberical column has
num_cols = df.select_dtypes(include=np.number).columns
outlier_counts = {}

for col in num_cols:
    mean = df[col].mean()
    std = df[col].std()

    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std

    outliers = ((df[col] < lower_bound) | (df[col] > upper_bound))
    outlier_counts[col] = outliers.sum()

outlier_counts = pd.Series(outlier_counts).sort_values(ascending=False)
outlier_counts



,0
Bsmt_Half_Bath,170
Kitchen_AbvGr,130
Screen_Porch,105
BsmtFin_SF_2,91
Enclosed_Porch,79
Mas_Vnr_Area,59
Open_Porch_SF,49
Sale_Price,44
Wood_Deck_SF,41
Three_season_porch,35


#### Part B

In [68]:
#identify column that has more than 150 outliers
cols_to_winsorize = outlier_counts[outlier_counts > 150].index.tolist()
print(cols_to_winsorize)

#replace value of column that has more than 150 outliers
for col in cols_to_winsorize:
    df[col] = df[col].astype(float)   #convert to float

    mean = df[col].mean()
    std = df[col].std()
    upper_cap = mean + 3 * std

    df.loc[df[col] > upper_cap, col] = upper_cap

df[cols_to_winsorize].describe()



['Bsmt_Half_Bath']


,Bsmt_Half_Bath
count,2829.000000
mean,0.048064
std,0.190123
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,0.799848


## Q3: Address Skewed Variables

In [69]:
#Compute skewness for each numeric column
skewness_values = df[num_cols].skew()
skewness_values.sort_values(ascending=False)

#Identify highly right-skewed features (skew > 1.0)
high_skewness_features = skewness_values[skewness_values > 1.0]
high_skew_cols = high_skewness_features.index.tolist()
print(f"high_skewness_features:\n{high_skewness_features}")

#log transformation
log_transform_cols = [
    col for col in high_skew_cols
    if (df[col] > 0).all()]      #exclude column with zero values

for col in log_transform_cols:
    df[col] = np.log(df[col])

df[log_transform_cols].skew().sort_values(ascending=False)


high_skewness_features:
Lot_Area              12.916210
Mas_Vnr_Area           2.707823
BsmtFin_SF_2           4.116586
Total_Bsmt_SF          1.161243
First_Flr_SF           1.498070
Gr_Liv_Area            1.294928
Bsmt_Half_Bath         3.704006
Kitchen_AbvGr          4.300875
Wood_Deck_SF           1.871091
Open_Porch_SF          2.535678
Enclosed_Porch         3.994499
Three_season_porch    10.611318
Screen_Porch           3.886562
Pool_Area             16.641845
Misc_Val              21.665830
Sale_Price             1.762402
dtype: float64


,0
First_Flr_SF,0.062867
Gr_Liv_Area,0.029453
Sale_Price,0.002280
Lot_Area,-0.471389


## Q4 Dummy Encoding Categorical Variables

### Part A

In [70]:
# Count number of categories per categorical column
category_counts = df[cat_cols].nunique().sort_values(ascending=False)

category_counts


,0
Neighborhood,28
Exterior_2nd,17
MS_SubClass,16
Exterior_1st,16
Sale_Type,10
House_Style,9
Condition_1,9
Overall_Cond,9
Functional,8
Roof_Matl,8


### Part B & C

In [71]:
#encoding option 3
num_cols_before = df.shape[1]

#Apply dummy encoding to the entire DataFrame
df_encoded = pd.get_dummies(df, drop_first=True)

num_cols_after = df_encoded.shape[1]
num_cols_after

print(f"Number of columns before dummy encoding: {num_cols_before}")
print(f"Number of columns after dummy encoding: {num_cols_after}")




Number of columns before dummy encoding: 72
Number of columns after dummy encoding: 268


In [72]:

from feature_engine.encoding import RareLabelEncoder

# Initialize the encoder
# tol=0.05 means categories with less than 5% frequency will be grouped
# n_categories=4 means it only applies to columns with more than 4 unique categories

# Columns BEFORE encoding
cols_before = df.shape[1]
encoder = RareLabelEncoder(tol=0.05, n_categories=4, replace_with='Other')

df_opt1 = encoder.fit_transform(df)

# Dummy encode after grouping rare labels
df_opt1_encoded = pd.get_dummies(df_opt1, drop_first=True)

cols_after = df_opt1_encoded.shape[1]

print(f"Number of columns before dummy encoding: {cols_before}")
print(f"Number of columns after dummy encoding: {cols_after}")

Number of columns before dummy encoding: 72
Number of columns after dummy encoding: 134


/usr/local/lib/python3.12/dist-packages/feature_engine/encoding/rare_label.py:216: UserWarning: The number of unique categories for variable Street is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/feature_engine/encoding/rare_label.py:216: UserWarning: The number of unique categories for variable Alley is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/feature_engine/encoding/rare_label.py:216: UserWarning: The number of unique categories for variable Lot_Shape is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/feature_engine/encoding/rare_label.py:216: UserWarning: The number of unique categories for variable Land_Contour is less than that indicated in n_categories. Thus, all categories w

In [73]:
#option 2
from sklearn.preprocessing import OneHotEncoder

cols_before = df.shape[1]

encoder = OneHotEncoder(drop="first", max_categories=5,sparse_output=False,handle_unknown="ignore")

encoded_array = encoder.fit_transform(df[cat_cols])
encoded_df = pd.DataFrame(
    encoded_array,
    columns=encoder.get_feature_names_out(cat_cols),
    index=df.index
)

# Combine with numeric columns
df_opt2_encoded = pd.concat(
    [df.drop(columns=cat_cols), encoded_df],
    axis=1
)

cols_after = df_opt2_encoded.shape[1]

print(f"Number of columns before dummy encoding: {cols_before}")
print(f"Number of columns after dummy encoding: {cols_after}")

cols_before, cols_after


Number of columns before dummy encoding: 72
Number of columns after dummy encoding: 169


(72, 169)

## Q5 Principal Component Analysis

### Part A

In [74]:
pca_features = ['Lot_Frontage', 'Lot_Area', 'Year_Built', 'Year_Remod_Add', 'Mas_Vnr_Area',

    'BsmtFin_SF_1', 'BsmtFin_SF_2', 'Bsmt_Unf_SF', 'Total_Bsmt_SF', 'First_Flr_SF',

    'Second_Flr_SF', 'Gr_Liv_Area', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath', 'Full_Bath',

    'Half_Bath', 'Bedroom_AbvGr', 'Kitchen_AbvGr', 'TotRms_AbvGrd', 'Fireplaces',

    'Garage_Cars', 'Garage_Area', 'Wood_Deck_SF', 'Open_Porch_SF', 'Enclosed_Porch',

    'Three_season_porch', 'Screen_Porch', 'Pool_Area', 'Misc_Val']

X = df[pca_features].copy()
#Transform all variables into standard form
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled

array([[ 2.66644708,  2.49991423, -0.36143354, ..., -0.28742331,
        -0.06415089, -0.08929675],
       [ 0.71300424,  0.52694938, -0.32857383, ...,  1.86744219,
        -0.06415089, -0.08929675],
       [ 0.7450279 ,  0.92924256, -0.42715295, ..., -0.28742331,
        -0.06415089, 21.61991259],
       ...,
       [ 0.13657849,  0.31670956,  0.69007706, ..., -0.28742331,
        -0.06415089,  1.12641897],
       [ 0.61693328,  0.23400274,  0.09860235, ..., -0.28742331,
        -0.06415089, -0.08929675],
       [ 0.52086233,  0.15746193,  0.72293676, ..., -0.28742331,
        -0.06415089, -0.08929675]])

### Part B

In [75]:
#run pca with 5 dimensions
from sklearn.decomposition import PCA

pca_5 = PCA(n_components=5)
X_pca_5 = pca_5.fit_transform(X_scaled)

explained = pca_5.explained_variance_ratio_
print("\nExplained variance ratio (PCA1, .. PCA 5):", explained)
print("Explained variance PC1:", explained[0])
print("Cumulative explained variance for all dimensions:", explained.sum())



Explained variance ratio (PCA1, .. PCA 5): [0.20396686 0.10212673 0.07008315 0.06600864 0.04367209]
Explained variance PC1: 0.20396685727101946
Cumulative explained variance for all dimensions: 0.48585745866054114


### Part C

In [76]:
# Initialize PCA with 0.80 (80% variance)

pca_80 = PCA(n_components=0.80)

# Fit on the scaled data

X_pca_80 = pca_80.fit_transform(X_scaled)

print(f"Number of components selected automatically: {pca_80.n_components_}")

Number of components selected automatically: 15


## Download Cleaned Dataset

In [77]:
df.to_csv('real_estate_market_data.csv', index=False)
files.download('real_estate_market_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>